# Cycle Finding

## Cycles in Astronomy

In this section, we present some data from a variable star.  The data consists of observed brightness ([magnitude](https://en.wikipedia.org/wiki/Apparent_magnitude)) as a function date (expressed as a [barycentric Julian date](https://en.wikipedia.org/wiki/Barycentric_Julian_Date) (`bjd`)).  Unfortunately, the data is neither uniformly space in time, nor complete (due to cloudy weather).

Assume that the brightness of the variable star is modelled by a periodic (but not nessesarily sinusoidal) function.  What is the period of oscillation?

*(Data courtesy of Michael Allen.)*

In [ ]:
%matplotlib inline
import numpy as np, matplotlib.pyplot as plt

bjd, magnitude, magnitude_err = np.loadtxt("data/V1_calibExcel.csv", delimiter=",").T
fig, ax = plt.subplots(figsize=(10, 5))
ax.errorbar(bjd, magnitude, yerr=magnitude_err, fmt="+")
ax.set(
    xlabel="Time (days, BJD)",
    ylabel="Brightness (magnitude)",
    title="Apparent magnitude of a variable star over time.",
);

## Temperature in Moab UT

Here we look for cycles in the temperature of Arches National Park near Moab UT, ordered from [NOAA](https://www.ncdc.noaa.gov/cdo-web/datasets):

```
ORDER ID: #2478833
https://www.ncdc.noaa.gov/cdo-web/orders?email=michael.forbes@gmail.com&id=2478833
Stations	
GHCND:USC00420336
Begin Date	1980-06-01 00:00
End Date	2021-01-29 23:59
Data Types	
TMAX TMIN TOBS
Units	
Metric
Custom Flag(s)	
Station Name
Eligible for Certification	No
```

We load the data using [Pandas](https://pandas.pydata.org/).

The data here has a `DATE` and three temperatures `TOBS`, `TMIN`, and `TMAX`.  The latter two represent the minimum and maximum temperatures over the day.  *(`TOBS` is the "observed temperature", but this dataset does not list the times, so I would probably stick to analyzing `TMIN` and `TMAX`.)*

From this data, can you determine how long is a [tropical year](https://en.wikipedia.org/wiki/Tropical_year) is?  With what accuracy?  How does your result/error estimate change if you include only a subset of the data?



In [ ]:
%matplotlib inline
import numpy as np, matplotlib.pyplot as plt
import pandas as pd

datafile = 'data/2478833.csv'
data = pd.read_csv(datafile, quotechar='"', parse_dates=[2])

In [ ]:
years = np.unique(data.DATE.dt.year)
decades = np.unique(years // 10)


fig, axs = plt.subplots(len(decades), 1, figsize=(12, 3 * len(decades)))
for decade, ax in zip(decades, axs):
    inds = np.where(data.DATE.dt.year // 10 == decade)[0]
    date, t0, tmin, tmax = (
        data.DATE[inds],
        data.TOBS[inds],
        data.TMIN[inds],
        data.TMAX[inds],
    )
    ax.errorbar(
        date,
        t0,
        yerr=[t0 - tmin, tmax - t0],
        ls="none",
        fmt="",
        elinewidth=0.5,
        capsize=2,
        capthick=1,
        alpha=0.5
    )
    ax.set(
        xlim=np.array([str(decade * 10), str((decade + 1) * 10)], dtype=np.datetime64),
        ylabel="$T$ [C]",
    )

To make this a little easier to work with, we convert the date to a time difference -- the number of days since the start of the observations -- and package them in a dictionary with three keys `TOBS`, `TMIN`, and `TMAX`

In [ ]:
from collections import namedtuple
t = (data.DATE - data.DATE[0]).dt.days.values
TimeTemp = namedtuple('TimeTemp', ['t', 'T'])
tTOBS, tTMIN, tTMAX = [TimeTemp(t[_inds], _d.values[_inds])
                       for _d in (data.TOBS, data.TMIN, data.TMAX)
                       for _inds in [np.where(~np.isnan(_d.values))[0]]]
tTs = dict(TOBS=tTOBS, TMIN=tTMIN, TMAX=tTMAX)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20,2), sharex=True)
for key in tTs:
    t, T = tTs[key]
    ax.plot(t, T, lw=0.2, alpha=0.5, label=key)
ax.set(xlabel='day', ylabel='$T$ [C]')
ax.legend()